In [ ]:
import spacy
from spacy.lang.en import English
import json
import string
import time
import random
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
import pandas as pd
from Bio import Entrez
from spacy.gold import GoldParse
from spacy.scorer import Scorer

In [ ]:
def load_data(file):
    with open(file) as f:
      data = json.load(f)
    return(data)

def save_data(file, data):
    with open(file, 'w', encoding="utf-8") as f:
        json.dump(data, f, indent = 4)

def process_data(data):
    start = time.time()
    my_list = []
    for j in range(0,len(data)):
        for i in range(0,len(data[j]['ingredients'])):
            my_list.append(data[j]['ingredients'][i]['text'])

    end = time.time()
    print("Time: ", end-start)
    return my_list

def new_clean_proc(text):
    start = time.time()
    print("1", text)
    text = [x for x in text if not any(c.isdigit() for c in x)]
    print("2", text)
    result = []
    for i in range(0,len(text)):
    
        pro_text = [char.lower() for char in text[i] if char not in string.punctuation]
        pro_text = ''.join(pro_text)
        no_stop_pro_text = [word for word in pro_text.split() if word.lower() not in stopwords.words('english')]
        no_stop_pro_text = ' '.join(no_stop_pro_text)
        result.append(no_stop_pro_text)
        print("3", no_stop_pro_text)
        

    end = time.time()
    print("Time: ", end-start)
    return result

In [ ]:
data = load_data('det_ingrs.json')
food = process_data(data)
print(len(food))

clean_food = new_clean_proc(food)
print('Rozmiar: ', len(clean_food))
print('Roznica: ', len(food) - len(clean_food))


In [ ]:
def division(mylist, number):
    five_list = []
    one_word = []
    for i in range(0,len(mylist)):
        if ((len(mylist[i].split()) <= number) and (len(mylist[i].split()) > 1)):
            five_list.append(mylist[i])
        if len(mylist[i].split()) == 1:
            one_word.append(mylist[i])
    return five_list, one_word

def separate(mylist):
    result = []
    for i in range(0,len(mylist)):
        mylist[i] = mylist[i].split()
        for j in range(0,len(mylist[i])):
            result.append(mylist[i][j])
    return result

def if_none(list):
    result = []
    for i in range(0,len(list)):
        if list[i]:
            result.append(list[i])
    return result

    
def if_food(word):
    syns = wn.synsets(str(word), pos = wn.NOUN)
    for syn in syns:
        if 'food' in syn.lexname():
            return word

In [ ]:
clean_food = list(set(clean_food))

max_five_list, one_word = division(clean_food, 5)
sep_food = separate(clean_food)
print("Liczba podzielonych encji: ", len(sep_food))

one_word_food = sep_food + one_word
one_word_food = list(set(one_word_food))
print("Liczba wszystkich jednowyrazowych encji: ", len(one_word_food))

one_word_finally = if_none(list(map(if_food, one_word_food)))
print("Koncowa liczba jednowyrazowych encji: ", len(one_word_finally))

clean_food_after = one_word_finally + max_five_list
print("Koncowy rozmiar danych: ", len(clean_food_after))

etykiety = load_data("etykiety.json")
print("Liczba encji to usunięcia: ", len(etykiety))

one_word_finally = list(filter(lambda x: x not in etykiety, one_word_finally))
print("Po usunieciu: ", len(one_word_finally))

clean_food_after = one_word_finally + max_five_list
print("Koncowy rozmiar danych: ", len(clean_food_after))

save_data('food_finally.json', clean_food_after)


In [ ]:
def search(query, number):
    Entrez.email = 'your.email@example.com'
    handle = Entrez.esearch(db='pubmed',
                            sort='relevance',
                            retmax= number,
                            retmode='xml',
                            term=query)
    results = Entrez.read(handle)
    return results

def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'your.email@example.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

def get_abstract(words, number, filename):
    start = time.time()
    article = []
    list_nr_id = []
    
    for word in words:
        print("Słowo:",word)
        no_abstr = 0
        proby = 0
        yes_abstr = 0
        results = search(word, number)
        id_list = results['IdList']
        papers = fetch_details(id_list)
        for i, paper in enumerate(papers['PubmedArticle']):
            proby+=1
            try:
                if paper['MedlineCitation']['Article']['Abstract']['AbstractText'] != None:
                    article.append(paper['MedlineCitation']['Article']['Abstract']['AbstractText'][0])
                    list_nr_id.append(paper['MedlineCitation']['PMID'])
                    yes_abstr += 1
            except:
                no_abstr += 1
    
        print("Podjęto prób ze słowem {}:".format(word), proby)
        print("Brak abstraktu ze słowem {}:".format(word), no_abstr)
        print("Ilość abstraktów ze słowem {}:".format(word), yes_abstr)
    print("Rozmiar zbioru abstaktów: ", len(article))
    final = []
    for i in range(len(article)):
        final.append({'text': article[i], 'id': list_nr_id[i]})

    save_data(filename, final)
    

    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print("Time:", "{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

    return article


def delete_duplicate(data):
    seen = set()
    result = []
    for d in data:
        t = tuple(d.items())
        #print(t[0])
        if t[0] not in seen:
            seen.add(t[0])
            result.append(d)

    return result

In [ ]:
list_food = ['food', 'foods', 'vegetables', 'vegetable', "fruit", "fruits", "grains", "meat","meats", "nutriment", "nutriments", "dietary", "dietaries", "dairy", "drink", "drinks", 'nutritious', 'dish', 'diet' , 'diets', 'cook', 'meal', 'meals', 'feed', 'nourishment', "groceries", "grill",  "alimentary", "aliment", "pabulum", "cud", 'allergy', 'allergies', 'spices', "spice", "recipe", "recipes"]
abstr = get_abstract(list_food, 10000, 'last_list_abstract.json')

abstr_list_data = load_data('last_list_abstract.json')
res = delete_duplicate(abstr_list_data)
save_data('last_abstract.json', res)

In [ ]:
def creating_data_training(data, type):
    patterns = []
    for item in data:
        pattern = {
                    "label": type,
                    "pattern": item,
        }
        patterns.append(pattern)
    return(patterns)

def generate_rules(patterns):
    nlp = English()
    ruler = nlp.add_pipe("food_finally")
    ruler.add_patterns(patterns)
    nlp.to_disk("patterns_food")

In [ ]:
patterns = creating_data_training(food, "FOOD")
generate_rules(patterns)

In [ ]:
def test_model(model, text):
    numb_ent = 0
    nlp = model
    doc = nlp(text)
    results = []
    entities = []
    for ent in doc.ents:
        entities.append((ent.start_char, ent.end_char, ent.label_))
        numb_ent = numb_ent + 1
    if len(entities) > 0:
        results = [text, {'entities': entities}]
    return (results, numb_ent)


def create_train_data(nlp_model, name_save_data, article):
    start = time.time()

    fin_numb_ent = 0
    TRAIN_DATA = []
    nlp = spacy.load(nlp_model)

    for i in range(len(article)):
        try:
                
                (result, numb_ent) = test_model(nlp, article[i])
                fin_numb_ent = numb_ent + fin_numb_ent

                if result != []:
                    TRAIN_DATA.append(result)
 
        except:
            continue
    print('Wielkosc zbioru traningowego: ', len(TRAIN_DATA))
    print('Liczba wykrytych encji: ', fin_numb_ent)
    save_data(name_save_data, TRAIN_DATA)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print("Time:", "{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [ ]:
abstract_data = load_data("abstract_data.json")
TRAIN_DATA = create_train_data('patterns_food', 'test_food_finally.json ', abstract_data)

In [ ]:
def train_food_spacy(TRAIN_DATA, iteration):
    start = time.time()
    nlp = spacy.blank("en")
    ner = nlp.create_pipe("ner")
    ner.add_label("FOOD")
    nlp.add_pipe(ner, name="test_food_ner")
    c = 0

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "test_food_ner"]
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(iteration):
            start_part = time.time()
            print("Start iteration: ", itn)
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text,annotations in TRAIN_DATA:
                nlp.update(
                            [text],
                            [annotations],
                            drop = 0.2,
                            sgd = optimizer,
                            losses = losses
                )
            end_part = time.time()
            hours, rem = divmod(end_part-start_part, 3600)
            minutes, seconds = divmod(rem, 60)
            print("Time of iteration ",itn,": ","{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print("Time:", "{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    return(nlp)

In [ ]:
random.shuffle(TRAIN_DATA)
nlp = train_food_spacy(TRAIN_DATA, 30)
nlp.to_disk("test_food_ner_model")

In [ ]:
def evaluate(nlp, examples):
    ner_model = spacy.load(nlp)
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    
    precision = scorer.scores['ents_p']
    recall = scorer.scores['ents_r']
    f1 = scorer.scores['ents_f']

    print("Model: ", nlp)
    print(scorer.scores)
    print()
    print("Precision: {:.2f}%".format(precision))
    print("Recall: {:.2f}%".format(recall))
    print("F1: {:.2f}%".format(f1))
    


def get_data(data):
    examples = []
    for i in range(len(data)):
        spannn = []
        for j in range(len(data[i]['spans'])):
            spannn.append((data[i]['spans'][j]['start'], data[i]['spans'][j]['end'], data[i]['spans'][j]['label']))
        examples.append((data[i]['text'], spannn))
    return examples

In [ ]:
gold_standard = load_data('annoted_food.json')
data = get_data(gold_standard)

evaluate("food_ner_model", data)
evaluate("test_food_ner_model", data)

In [ ]:
def get_food(df, nlp):
    j=0
    c=0
    a=10
    text = df['text']
    df['food'] = 0
    df['count_f'] = 0
    for i in range(len(text)): 
        j+=1
        if round(j*100 / len(text)) == a:
            print("---------------- Progress: {:.0f}% ----------------".format(a)) 
            a +=10
        lists = []
        doc = nlp(text[i])
        for ent in doc.ents:
            lists.append(ent.text)
        if lists != []:
            df['food'][i] = lists
            df['count_f'][i] = len(lists)

    return df

nlp = spacy.load("test_food_ner_model")

In [ ]:
result = load_data("wyniki_z_bertowego_modelu.json")
df = pd.DataFrame(result, columns = ['text', 'id'])
df_food = get_food(df, nlp)

In [ ]:
def get_med(df, nlp):
    j=0
    c=0
    a=10
    text = df['text']
    df['med'] = 0
    df['count_m'] = 0
    for i in range(len(text)): 
        j+=1
        if round(j*100 / len(text)) == a:
            print("---------------- Progress: {:.0f}% ----------------".format(a)) 
            a +=10
        lists = []
        doc = nlp(text[i])
        for ent in doc.ents:
            if ent.label_ == "DISEASE":
                lists.append(ent.text)
        if lists != []:
            df['med'][i] = lists
            df['count_m'][i] = len(lists)
    return df

nlp = spacy.load("en_ner_bc5cdr_md")

In [ ]:
df_med = get_med(df_food, nlp)
df_med.to_csv('bertowy_model_z_food_med.csv', index = False, header = True)